In [1]:
import os
import numpy as np
import argoverse
import shutil
from argoverse.data_loading.argoverse_tracking_loader import ArgoverseTrackingLoader
from argoverse.utils.ply_loader import load_ply
from argoverse.utils.camera_stats import get_image_dims_for_camera as get_dim

In [2]:
root_dir = './../data/'
trains = ['train1/', 'train2/', 'train3/', 'train4/']
subroot_dir = root_dir + 'train1/'
disparity_dir = root_dir + 'disparity/'
stereo_left_dir = root_dir + 'stereo_left/'
stereo_right_dir = root_dir + 'stereo_right/'

In [3]:
argoverse_loader = ArgoverseTrackingLoader(subroot_dir)

In [4]:
print('Total number of logs:',len(argoverse_loader))

Total number of logs: 3


In [5]:
argoverse_loader.print_all()

#images: 474
#lidar: 471


In [6]:
print(argoverse_loader.log_list)

['8a15674a-ae5c-38e2-bc4b-f4156d384072', '5ab2697b-6e3e-3454-a36a-aba2c6f27818', '1d676737-4110-3f7e-bec0-0c90f74c248f']


In [7]:
camL = argoverse_loader.CAMERA_LIST[7] # left stereo
camR = argoverse_loader.CAMERA_LIST[8] # right stereo
(width, height) = get_dim(camL)
baseline = 0.3

In [9]:
for log_id in argoverse_loader.log_list:
    print("processing log:", log_id)
    get_log_identifier = log_id.split('-')[0]
    argoverse_data = argoverse_loader.get(log_id)
    calibL = argoverse_data.get_calibration(camL)
    calibR = argoverse_data.get_calibration(camR)
    focalX_px = calibL.K[0,0]
    #print(focalX_px)
    baseline_focal = focalX_px * baseline
    
    if not os.path.isdir(disparity_dir):
        os.makedirs(disparity_dir)
    if not os.path.isdir(stereo_left_dir):
        os.makedirs(stereo_left_dir)
    if not os.path.isdir(stereo_right_dir):
        os.makedirs(stereo_right_dir)
        
    lidar_list = argoverse_data.lidar_list
    lidar_timestamp_list = argoverse_data.lidar_timestamp_list
    
    stereo_left_list = argoverse_data.image_list
    stereo_left_timestamp_list = argoverse_data.image_timestamp_list['stereo_front_left']
    stereo_right_timestamp_list = argoverse_data.image_timestamp_list['stereo_front_right']
    
    # get the first corresponding timestamp
    lidar_timeonly = [str(a)[:10] for a in lidar_timestamp_list]
    stereo_timeonly = [str(a)[:10] for a in stereo_left_timestamp_list]
    
    overlap = []
    for i in range(len(stereo_timeonly)):
        for j in range(len(lidar_timeonly)):
           if(lidar_timeonly[j] == stereo_timeonly[i]):
                overlap.append(stereo_timeonly[i])
    
    # sort only corresponding lidar files
    only_lidar = []
    for i in range(len(overlap)):
        for j in range(len(lidar_list)):
            if(overlap[i] in lidar_list[j]):
                only_lidar.append(lidar_list[j])
                
    only_lidar_time = []
    for i in range(len(overlap)):
        for j in range(len(lidar_timestamp_list)):
            if(overlap[i] in str(lidar_timestamp_list[j])):
                only_lidar_time.append(lidar_timestamp_list[j])
                
    # sort only corresponding stereo left files
    only_left = []
    for i in range(len(overlap)):
        for j in range(len(stereo_left_list['stereo_front_left'])):
            if(overlap[i] in stereo_left_list['stereo_front_left'][j]):
                only_left.append(stereo_left_list['stereo_front_left'][j])
                
    # sort only corresponding stereo right files
    only_right = []
    for i in range(len(overlap)):
        for j in range(len(stereo_left_list['stereo_front_right'])):
            if(overlap[i] in stereo_left_list['stereo_front_right'][j]):
                only_right.append(stereo_left_list['stereo_front_right'][j])
                
    for idx in range(len(only_lidar)):
        lidar_timestamp = only_lidar_time[idx]
        print("index: ", idx, "current timestamp: ", lidar_timestamp)
        pc = load_ply(only_lidar[idx])
        
        uv = calibL.project_ego_to_image(pc)
        
        fov_inds = (uv[:, 0] < width - 1) & (uv[:, 0] >= 0) & \
                   (uv[:, 1] < height - 1)& (uv[:, 1] >= 0)
        fov_inds = fov_inds & (pc[:, 0] > 1) # filters out points that are behind camera
        
        valid_uv = uv[fov_inds, :]  
        valid_uv = np.round(valid_uv).astype(int)
        
        valid_pc = pc[fov_inds, :]        
        valid_uvd = calibL.project_ego_to_cam(valid_pc)
        
        depth_map = np.zeros((height, width)) - 1
        
        for i in range(valid_uv.shape[0]):
            depth_map[int(valid_uv[i, 1]), int(valid_uv[i, 0])] = valid_uvd[i, 2]
        
        disp_map = (calibL.K[0,0] * baseline) / depth_map
        print(disp_map.shape)
        
        normalized_disp_map = disp_map
        normalized_disp_map[normalized_disp_map < 0] = -1.0
        
        print(normalized_disp_map.shape)
        disp_store_name = get_log_identifier + '_' + str(lidar_timestamp)
        np.save(disparity_dir + disp_store_name, disp_map)
        
    #save corresponding left and right images
    for i in range(len(overlap)):
        
        get_left_name = only_left[i].split('/')[-1]
        get_right_name = only_right[i].split('/')[-1]
        
        left_store_name = get_log_identifier + '_' + get_left_name
        right_store_name = get_log_identifier + '_' + get_right_name
        
        shutil.copy2(only_left[i], stereo_left_dir + left_store_name)
        shutil.copy2(only_right[i], stereo_right_dir + right_store_name)
        
    print("complete")

processing log: 8a15674a-ae5c-38e2-bc4b-f4156d384072
index:  0 current timestamp:  315969684120175000
(2056, 2464)
(2056, 2464)
index:  1 current timestamp:  315969684319904000
(2056, 2464)
(2056, 2464)
index:  2 current timestamp:  315969684520297000
(2056, 2464)
(2056, 2464)
index:  3 current timestamp:  315969684720026000
(2056, 2464)
(2056, 2464)
index:  4 current timestamp:  315969684919755000
(2056, 2464)
(2056, 2464)
index:  5 current timestamp:  315969685119482000
(2056, 2464)
(2056, 2464)
index:  6 current timestamp:  315969685319875000
(2056, 2464)
(2056, 2464)
index:  7 current timestamp:  315969685519604000
(2056, 2464)
(2056, 2464)
index:  8 current timestamp:  315969685719333000
(2056, 2464)
(2056, 2464)
index:  9 current timestamp:  315969685919726000
(2056, 2464)
(2056, 2464)
index:  10 current timestamp:  315969686119335000
(2056, 2464)
(2056, 2464)
index:  11 current timestamp:  315969686319728000
(2056, 2464)
(2056, 2464)
index:  12 current timestamp:  31596968652012

(2056, 2464)
(2056, 2464)
index:  29 current timestamp:  315972995819766000
(2056, 2464)
(2056, 2464)
index:  30 current timestamp:  315972996019497000
(2056, 2464)
(2056, 2464)
index:  31 current timestamp:  315972996219890000
(2056, 2464)
(2056, 2464)
index:  32 current timestamp:  315972996419619000
(2056, 2464)
(2056, 2464)
index:  33 current timestamp:  315972996619349000
(2056, 2464)
(2056, 2464)
index:  34 current timestamp:  315972996819742000
(2056, 2464)
(2056, 2464)
index:  35 current timestamp:  315972997018968000
(2056, 2464)
(2056, 2464)
index:  36 current timestamp:  315972997219361000
(2056, 2464)
(2056, 2464)
index:  37 current timestamp:  315972997419754000
(2056, 2464)
(2056, 2464)
index:  38 current timestamp:  315972997620147000
(2056, 2464)
(2056, 2464)
index:  39 current timestamp:  315972997819877000
(2056, 2464)
(2056, 2464)
index:  40 current timestamp:  315972998020014000
(2056, 2464)
(2056, 2464)
index:  41 current timestamp:  315972998219744000
(2056, 2464)

(2056, 2464)
(2056, 2464)
index:  58 current timestamp:  315984819620307000
(2056, 2464)
(2056, 2464)
index:  59 current timestamp:  315984819820036000
(2056, 2464)
(2056, 2464)
index:  60 current timestamp:  315984820020063000
(2056, 2464)
(2056, 2464)
index:  61 current timestamp:  315984820219792000
(2056, 2464)
(2056, 2464)
index:  62 current timestamp:  315984820419522000
(2056, 2464)
(2056, 2464)
index:  63 current timestamp:  315984820619251000
(2056, 2464)
(2056, 2464)
index:  64 current timestamp:  315984820819644000
(2056, 2464)
(2056, 2464)
index:  65 current timestamp:  315984821020040000
(2056, 2464)
(2056, 2464)
index:  66 current timestamp:  315984821219769000
(2056, 2464)
(2056, 2464)
index:  67 current timestamp:  315984821420162000
(2056, 2464)
(2056, 2464)
index:  68 current timestamp:  315984821619891000
(2056, 2464)
(2056, 2464)
index:  69 current timestamp:  315984821820284000
(2056, 2464)
(2056, 2464)
index:  70 current timestamp:  315984822020016000
(2056, 2464)